In [1]:
from bs4 import BeautifulSoup
import re

In [13]:
def split_tag(tag):
  """Split a tag to get the name of the actual tag"""
  try:
    split_tag = lambda tag, point : tag.split(point)
    point_list = ['#', '.', ' ', '\n', '\t']
    finder = [9999,'']
    for index, p in enumerate(point_list):
      pos = tag.find(p)
      if pos < finder[0] and pos > 0:
        finder[0] = pos
        finder[1] = p
    return tag.split(finder[1], 1)[0]
  except:
    return tag

In [14]:
def get_class_id(element, type):
  """Returns class or id attribute in proper HTML form"""
  attr = 'class' if type == '\.' else 'id'
  matches = re.search(f"({type}+\w+\S+)", element)
  if matches is not None:
    split_del = '.' if type =='\.' else type
    attrs = matches.group(0).split(split_del)
    new_element = element.replace(matches.group(0), '')
    return f"{attr}=\"{' '.join(attrs[1:])}\"", new_element
  return None, element

def get_attributes(element):
  """
    Get the attributes of the elements.
    1. Get classes
    2. Get id
    3. Other attrs
  """
  combined_attrs = ''
  classes, new_el = get_class_id(element, '\.')
  id, _ = get_class_id(new_el, '#')
  if classes is not None:
    combined_attrs += classes
  
  if id is not None:
    combined_attrs += id

  return combined_attrs

In [15]:
def get_childs(x, i, t, end=""):
  """Recursive function that gets the child elements from a starting position"""
  if i >= len(x):
    return ""
  tabs_count = x[i].count('\t')
  xi = x[i]
  attrs = get_attributes(xi)
  attrs = ' ' + attrs
  xi = split_tag(x[i])
  
  if tabs_count > t or i > len(x):
    h = get_childs(x, i+1, tabs_count, '</'+xi+'>')
    #xi = split_tag(x[i])
    return '<'+xi+attrs+'>' + h + end
  elif tabs_count == t:
    h = get_childs(x, i+1, tabs_count, end)
    return '<'+xi+attrs+'></'+xi+'>' + h
  elif tabs_count < t:
    if i+1 <= len(x):
      h = get_childs(x, i+1, tabs_count)
    else:
      h = ""
    return '<'+xi+attrs+'></'+xi+'>' + end + h
  else:
    return ""

In [16]:
def init_string(string):
  string = string.replace('  ', '\t')
  string = string.split(':')
  return string

def get_pretty_html(string):
  pcml = get_childs(init_string(string), 1, 0)
  return BeautifulSoup(pcml, 'html.parser').prettify()

In [17]:
test_string = '''
:div
:main.vmainClass1.vmainClass2
  :div#main-container
    :div#keyboard-shortcut.class2
    :div
    :div
  :div.form-container
    :form method=post
  :v-icon
    :v-con'''

print(get_pretty_html(test_string))

<div>
</div>
<main class="vmainClass1 vmainClass2">
 <div id="main-container">
  <div class="class2" id="keyboard-shortcut">
  </div>
  <div>
  </div>
  <div>
  </div>
 </div>
 <div class="form-container">
  <form>
  </form>
 </div>
 <v-icon>
  <v-con>
  </v-con>
 </v-icon>
</main>
